# Day 2 - Lab 1: AI-Generated System Design & Database Seeding (Solution)

**Objective:** Use the PRD artifact from Day 1 to generate a detailed SQL database schema, create realistic seed data, and then use those outputs to create and seed a live, local database file.

**Introduction:**
This solution notebook provides the complete prompts and Python code for Day 2's first lab. It demonstrates the workflow of generating design artifacts (SQL schema, seed data) and then using code to create a physical database from them.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

**Explanation:**
We load the `day1_prd.md` artifact from Day 1. This document is the single source of truth for our project's requirements and provides the essential context for the LLM to generate a relevant and accurate database schema.

In [1]:
import sys
import os
import sqlite3

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, recommended_models_table

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

# Load the PRD from Day 1
prd_content = load_artifact("artifacts/day1_prd.md")
if not prd_content:
    print("Warning: Could not load day1_prd.md. Lab may not function correctly.")

✅ LLM Client configured: Using 'openai' with model 'gpt-4o'


In [2]:
recommended_models_table()

| Model | Provider | Vision | Image Gen | Audio Transcription | Context Window | Max Output Tokens |
|---|---|---|---|---|---|---|
| claude-opus-4-1-20250805 | anthropic | ✅ | ❌ | ❌ | 200,000 | 100,000 |
| claude-opus-4-20250514 | anthropic | ✅ | ❌ | ❌ | 200,000 | 100,000 |
| claude-sonnet-4-20250514 | anthropic | ✅ | ❌ | ❌ | 1,000,000 | 100,000 |
| codex-mini-latest | openai | ✅ | ❌ | ❌ | 200,000 | 100,000 |
| dall-e-3 | openai | ❌ | ✅ | ❌ | - | - |
| deepseek-ai/DeepSeek-V3 | huggingface | ❌ | ❌ | ❌ | 128,000 | 100,000 |
| deepseek-ai/DeepSeek-V3-Small | huggingface | ❌ | ❌ | ❌ | 128,000 | 100,000 |
| deepseek-ai/DeepSeek-VL2 | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |
| deepseek-ai/DeepSeek-VL2-Small | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |
| deepseek-ai/DeepSeek-VL2-Tiny | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |
| deepseek-ai/Janus-Pro-7B | huggingface | ✅ | ❌ | ❌ | 8,192 | 2,048 |
| gemini-2.0-flash | google | ✅ | ❌ | ❌ | 1,048,576 | 8,192 |
| gemini-2.0-flash-lite | google | ✅ | ❌ | ❌ | 1,048,576 | 8,192 |
| gemini-2.0-flash-live-001 | google | ✅ | ❌ | ❌ | 1,048,576 | 8,192 |
| gemini-2.5-flash | google | ✅ | ❌ | ❌ | 1,048,576 | 65,536 |
| gemini-2.5-flash-image-preview | google | ✅ | ✅ | ❌ | 32,768 | 32,768 |
| gemini-2.5-flash-lite | google | ✅ | ❌ | ❌ | 1,048,576 | 65,536 |
| gemini-2.5-pro | google | ✅ | ❌ | ❌ | 1,048,576 | 65,536 |
| gemini-live-2.5-flash-preview | google | ✅ | ❌ | ❌ | 1,048,576 | 8,192 |
| gemini-veo-3 | google | ✅ | ❌ | ❌ | - | - |
| google-cloud/speech-to-text/latest_long | google | ❌ | ❌ | ✅ | - | - |
| google-cloud/speech-to-text/latest_short | google | ❌ | ❌ | ✅ | - | - |
| gpt-4.1 | openai | ✅ | ❌ | ❌ | 1,000,000 | 32,000 |
| gpt-4.1-mini | openai | ✅ | ❌ | ❌ | 1,000,000 | 32,000 |
| gpt-4.1-nano | openai | ✅ | ❌ | ❌ | 1,000,000 | 32,000 |
| gpt-4.5 | openai | ✅ | ❌ | ❌ | 128,000 | 16,384 |
| gpt-4o | openai | ✅ | ❌ | ❌ | 128,000 | 16,384 |
| gpt-4o-mini | openai | ✅ | ❌ | ❌ | 128,000 | 16,384 |
| gpt-5-2025-08-07 | openai | ✅ | ❌ | ❌ | 400,000 | 128,000 |
| gpt-5-mini-2025-08-07 | openai | ✅ | ❌ | ❌ | 400,000 | 128,000 |
| gpt-5-nano-2025-08-07 | openai | ✅ | ❌ | ❌ | 400,000 | 128,000 |
| gpt-image-1 | openai | ✅ | ✅ | ❌ | - | - |
| imagen-3.0-generate-002 | google | ❌ | ✅ | ❌ | - | - |
| imagen-4.0-generate-001 | google | ❌ | ✅ | ❌ | 480 | - |
| meta-llama/Llama-3.3-70B-Instruct | huggingface | ❌ | ❌ | ❌ | 4,096 | 1,024 |
| meta-llama/Llama-4-Maverick-17B-128E-Instruct | huggingface | ✅ | ❌ | ❌ | 1,000,000 | 100,000 |
| meta-llama/Llama-4-Scout-17B-16E-Instruct | huggingface | ✅ | ❌ | ❌ | 10,000,000 | 100,000 |
| mistralai/Mistral-7B-Instruct-v0.3 | huggingface | ❌ | ❌ | ❌ | 32,768 | 8,192 |
| o3 | openai | ✅ | ❌ | ❌ | 200,000 | 100,000 |
| o4-mini | openai | ✅ | ❌ | ❌ | 200,000 | 100,000 |
| tokyotech-llm/Llama-3.1-Swallow-70B-Instruct-v0.3 | huggingface | ❌ | ❌ | ❌ | 4,096 | 1,024 |
| tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.5 | huggingface | ❌ | ❌ | ❌ | 4,096 | 1,024 |
| whisper-1 | openai | ❌ | ❌ | ✅ | - | - |

'| Model | Provider | Vision | Image Gen | Audio Transcription | Context Window | Max Output Tokens |\n|---|---|---|---|---|---|---|\n| claude-opus-4-1-20250805 | anthropic | ✅ | ❌ | ❌ | 200,000 | 100,000 |\n| claude-opus-4-20250514 | anthropic | ✅ | ❌ | ❌ | 200,000 | 100,000 |\n| claude-sonnet-4-20250514 | anthropic | ✅ | ❌ | ❌ | 1,000,000 | 100,000 |\n| codex-mini-latest | openai | ✅ | ❌ | ❌ | 200,000 | 100,000 |\n| dall-e-3 | openai | ❌ | ✅ | ❌ | - | - |\n| deepseek-ai/DeepSeek-V3 | huggingface | ❌ | ❌ | ❌ | 128,000 | 100,000 |\n| deepseek-ai/DeepSeek-V3-Small | huggingface | ❌ | ❌ | ❌ | 128,000 | 100,000 |\n| deepseek-ai/DeepSeek-VL2 | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |\n| deepseek-ai/DeepSeek-VL2-Small | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |\n| deepseek-ai/DeepSeek-VL2-Tiny | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |\n| deepseek-ai/Janus-Pro-7B | huggingface | ✅ | ❌ | ❌ | 8,192 | 2,048 |\n| gemini-2.0-flash | google | ✅ | ❌ | ❌ | 1,048,576 | 8,192 |\n| gemini-2.0-fla

## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): Generating the SQL Schema

**Explanation:**
This prompt instructs the LLM to act as a Database Administrator (DBA). By providing the full PRD as context, we enable the LLM to understand the entities and relationships required by the application. The prompt specifically asks for `CREATE TABLE` statements, guiding the LLM to produce a ready-to-use SQL script. We then clean up the response to remove markdown fences and save the pure SQL code.

In [3]:
schema_prompt = f"""
You are an expert Database Administrator (DBA).

Based on the following Product Requirements Document (PRD), design a normalized SQL schema for a SQLite database. The schema should include tables for users and their assigned onboarding tasks.

**PRD Context:**
<prd>
{prd_content}
</prd>

The schema should have at least a `users` table and an `onboarding_tasks` table with a foreign key relationship.
- The `users` table should include an id, name, email, and role (e.g., 'New Hire', 'Manager').
- The `onboarding_tasks` table should include an id, a title, a description, a due_date, a status (e.g., 'Pending', 'Completed'), and a user_id foreign key.

Output only the raw SQL `CREATE TABLE` statements.
"""

print("--- Generating SQL Schema ---")
if prd_content:
    generated_schema = get_completion(schema_prompt, client, model_name, api_provider)
    
    # Clean up the generated schema
    cleaned_schema = clean_llm_output(generated_schema, language='sql')
    print(cleaned_schema)
    
    # Save the cleaned schema to a file
    save_artifact(cleaned_schema, "artifacts/schema.sql")
else:
    print("Skipping schema generation because PRD is missing.")
    cleaned_schema = ""

--- Generating SQL Schema ---
CREATE TABLE users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE,
    role TEXT CHECK(role IN ('New Hire', 'Manager', 'HR Manager', 'Team Leader')) NOT NULL
);

CREATE TABLE onboarding_tasks (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    description TEXT,
    due_date DATE,
    status TEXT CHECK(status IN ('Pending', 'Completed')) NOT NULL DEFAULT 'Pending',
    user_id INTEGER,
    FOREIGN KEY (user_id) REFERENCES users(id)
);
✅ Successfully saved artifact to: artifacts/schema.sql


### Challenge 2 (Intermediate): Generating Realistic Seed Data

**Explanation:**
An empty database isn't very useful for development. This prompt asks the LLM to generate realistic seed data. By providing both the PRD (for thematic context) and the SQL schema (for structural correctness), we guide the LLM to create `INSERT` statements that are both thematically appropriate (e.g., onboarding-related task titles) and syntactically correct.

In [4]:
seed_data_prompt = f"""
You are a data specialist. Based on the provided PRD and SQL schema, generate 5-10 realistic SQL `INSERT` statements to populate the tables with sample data for an onboarding tool.

**PRD Context:**
<prd>
{prd_content}
</prd>

**SQL Schema:**
<schema>
{cleaned_schema}
</schema>

Generate at least 3 users and 5 tasks assigned to those users.
Output only the raw SQL `INSERT` statements.
"""

print("--- Generating Seed Data ---")
if prd_content and cleaned_schema:
    generated_seed_data = get_completion(seed_data_prompt, client, model_name, api_provider)
    
    # Clean up the generated seed data
    cleaned_seed_data = clean_llm_output(generated_seed_data, language='sql')
    print(cleaned_seed_data)
    
    # Save the cleaned seed data to a file
    save_artifact(cleaned_seed_data, "artifacts/seed_data.sql")
else:
    print("Skipping seed data generation because PRD or schema is missing.")

--- Generating Seed Data ---
-- Insert sample users
INSERT INTO users (name, email, role) VALUES ('Alice Johnson', 'alice.johnson@example.com', 'New Hire');
INSERT INTO users (name, email, role) VALUES ('Bob Smith', 'bob.smith@example.com', 'HR Manager');
INSERT INTO users (name, email, role) VALUES ('Charlie Brown', 'charlie.brown@example.com', 'Team Leader');

-- Insert sample onboarding tasks for Alice Johnson (New Hire)
INSERT INTO onboarding_tasks (title, description, due_date, status, user_id) 
VALUES ('Complete Personal Details', 'Fill out personal details in the HR system', '2023-10-15', 'Pending', 1);

INSERT INTO onboarding_tasks (title, description, due_date, status, user_id) 
VALUES ('Read Company Handbook', 'Read and understand the company handbook', '2023-10-20', 'Pending', 1);

INSERT INTO onboarding_tasks (title, description, due_date, status, user_id) 
VALUES ('Complete Introduction Training', 'Complete the introductory training module', '2023-10-25', 'Pending', 1);

-

### Challenge 3 (Advanced): Creating and Seeding a Live Database

**Explanation:**
This Python function demonstrates a crucial engineering task: turning text-based artifacts into a live system component. The `create_database` function uses Python's built-in `sqlite3` library.
1.  It establishes a connection to a database file, which creates the file if it doesn't exist.
2.  It reads the `schema.sql` artifact and executes it. It's important to use `cursor.executescript()` here. While `cursor.execute()` is designed for a single SQL statement, `executescript()` is necessary for running a string that contains multiple SQL statements, which is exactly what our `schema.sql` and `seed_data.sql` files contain.
3.  It then reads and executes the `seed_data.sql` artifact to populate the newly created tables.
4.  `conn.commit()` saves all the changes to the database file.
5.  The `finally` block ensures that `conn.close()` is always called, which is a critical best practice to prevent resource leaks.

In [5]:
def create_database(db_path, schema_path, seed_path):
    """Creates and seeds a SQLite database from SQL files."""
    if not os.path.exists(schema_path):
        print(f"Error: Schema file not found at {schema_path}")
        return

    # Delete the old database file if it exists to start fresh
    if os.path.exists(db_path):
        os.remove(db_path)
        print(f"Removed existing database file at {db_path}")

    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        print(f"Successfully connected to database at {db_path}")

        # Read and execute the schema file
        schema_sql = load_artifact(schema_path)
        if schema_sql:
            cursor.executescript(schema_sql)
            print("Tables created successfully.")

        # Read and execute the seed data file if it exists
        if os.path.exists(seed_path):
            seed_sql = load_artifact(seed_path)
            if seed_sql:
                cursor.executescript(seed_sql)
                print("Seed data inserted successfully.")

        conn.commit()
        print("Database changes committed.")

    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")

# Define file paths
db_file = os.path.join(project_root, "artifacts", "onboarding.db")
schema_file = os.path.join(project_root, "artifacts", "schema.sql")
seed_file = os.path.join(project_root, "artifacts", "seed_data.sql")

# Execute the function
create_database(db_file, schema_file, seed_file)

Removed existing database file at /Users/agaleana/repos/AG-AISOFTDEV/artifacts/onboarding.db
Successfully connected to database at /Users/agaleana/repos/AG-AISOFTDEV/artifacts/onboarding.db
Tables created successfully.
Seed data inserted successfully.
Database changes committed.
Database connection closed.


## Lab Conclusion

Excellent work! You have now moved from abstract requirements to a concrete, physical database artifact. You've used an LLM to design a schema, generate realistic test data, and then used a Python script to bring that database to life. This `onboarding.db` file is the foundation upon which we will build our API in Day 3.

> **Key Takeaway:** The ability to generate structured data definitions (like a SQL schema) from unstructured text (like a PRD) is a core skill in AI-assisted development. It automates a critical and often time-consuming design step.